In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import imutils
import os
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
pathToExecutable = (
    "C:/Program Files/GNU Octave/Octave-7.1.0/mingw64/bin/octave-cli.exe"
)
os.environ['OCTAVE_EXECUTABLE'] = pathToExecutable
from oct2py import octave
from detect_face import detect_face, FaceAligner
fa = FaceAligner(desiredFaceWidth=224)

In [2]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
dir = os.path.dirname(os.getcwd())

In [3]:
missing_imgs= ['missing/010A06.JPG','missing/010A07b.JPG','missing/010A09.JPG','missing/015A00.JPG','missing/015A01.JPG','missing/015A03.JPG',
'missing/015A04.JPG','missing/067A14.JPG','missing/067A17.JPG',
'missing/067A21.JPG','missing/067A23.JPG','missing/071A22.JPG', 
'missing/071A24.JPG','missing/071A27.JPG','missing/071A35.JPG',
'missing/001A22.JPG','missing/001A28.JPG','missing/001A29.JPG',
'missing/001A33.JPG','missing/003A47.JPG','missing/003A49.JPG',
'missing/003A58.JPG','missing/011A15.JPG','missing/011A20.JPG',
'missing/011A30.JPG','missing/011A34.JPG',
'missing/WhatsApp_Image_2022-05-29_at_7.jpeg',
'missing/WhatsApp_Image_2022-05-292.jpeg',
'missing/WhatsApp_Image_2022-05-29_at_7.00.18_PM_2.jpeg',
'missing/WhatsApp_Image_2022-05-28_at_1.12.11_PM.jpeg',
'missing/WhatsApp_Image_2022-05-28_at_2.16.09_PM.jpeg',
'missing/WhatsApp_Image_2022-05-28_at_2.14.35_PM.jpeg',
'missing/IMG_20190404_194014.jpg','missing/IMG026.jpg',
'missing/Photo0149.jpg']
missing_labels= ['SanjanaTrivari_2007-03-04','SanjanaTrivari_2007-03-04','SanjanaTrivari_2007-03-04','AndyLilith_2002-09-09','AndyLilith_2002-09-09','AndyLilith_2002-09-09','AndyLilith_2002-09-09','AshinaRai_2013-09-04','AshinaRai_2013-09-04','AshinaRai_2013-09-04','AshinaRai_2013-09-04',
'ManishPaul_2019-09-22','ManishPaul_2019-09-22','ManishPaul_2019-09-22',
'ManishPaul_2019-09-22','AdamSingh_2016-02-28','AdamSingh_2016-02-28',
'AdamSingh_2016-02-28','AdamSingh_2016-02-28','SushaNair_2022-02-03',
'SushaNair_2022-02-03','SushaNair_2022-02-03','SurajSinha_2022-12-05',
'SurajSinha_2022-12-05','SurajSinha_2022-12-05','SurajSinha_2022-12-05',
'KeerthanaK_2022-05-22','KeerthanaK_2022-05-22','KeerthanaK_2022-05-22',
'AnsuiyaSharma_2021-05-10','AnsuiyaSharma_2021-05-10','AnsuiyaSharma_2021-05-10','KusumRamachandra_2013-03-04','KusumRamachandra_2013-03-04','KusumRamachandra_2013-03-04',]

In [4]:
#missing_labels.append('check')
n_labels = len(set(missing_labels))
print(n_labels)

10


In [5]:
len(missing_labels)

35

In [6]:

label_encoder = preprocessing.LabelEncoder()
labels_dict = dict()
u_labels=missing_labels
missing_labels_en= label_encoder.fit_transform(missing_labels)
for i in range(0,len(missing_labels)):
    if missing_labels_en[i] not in labels_dict:
        labels_dict[missing_labels_en[i]]=missing_labels[i]
        u_labels.append(missing_labels[i])
        print(missing_labels[i])

SanjanaTrivari_2007-03-04
AndyLilith_2002-09-09
AshinaRai_2013-09-04
ManishPaul_2019-09-22
AdamSingh_2016-02-28
SushaNair_2022-02-03
SurajSinha_2022-12-05
KeerthanaK_2022-05-22
AnsuiyaSharma_2021-05-10
KusumRamachandra_2013-03-04


In [7]:
labels_dict

{7: 'SanjanaTrivari_2007-03-04',
 1: 'AndyLilith_2002-09-09',
 3: 'AshinaRai_2013-09-04',
 6: 'ManishPaul_2019-09-22',
 0: 'AdamSingh_2016-02-28',
 9: 'SushaNair_2022-02-03',
 8: 'SurajSinha_2022-12-05',
 4: 'KeerthanaK_2022-05-22',
 2: 'AnsuiyaSharma_2021-05-10',
 5: 'KusumRamachandra_2013-03-04'}

In [8]:
missing_labels_en= label_encoder.fit_transform(u_labels)

In [9]:
model = tf.keras.applications.VGG16(
            include_top=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None,
            classes=1000,
            classifier_activation="softmax",
        )

In [10]:
def preprocess(image_list,aligned_img):
    #labels = np.zeros((1, len(missing_imgs)+1))
    i = 0
    images = []
    for filename in image_list:
        f=filename.split('/')
        img = cv2.imread(os.path.join(dir,'content','media',f[0],f[1]))
        img= cv2.equalizeHist(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
        
        img= cv2.resize(img,(224,224))
        img2 = cv2.merge((img,img,img))
        img = img_to_array(img2)
        images.append(img2)
        i +=1
    n_labels = len(set(missing_labels))
    for i in range(0,n_labels):
        img = cv2.equalizeHist(cv2.cvtColor(aligned_img, cv2.COLOR_BGR2GRAY))
            
        img= cv2.resize(img,(224,224))
        img2 = cv2.merge((img,img,img))
        img = img_to_array(img2)
        images.append(img)
    return np.array(images)

In [11]:
def feature_extraction(images):
    with tf.device('/device:cpu:0'):
        Flatten_list = [];
        FC7_list = [];
        FC6_list = [];
        l = len(model.layers)
        for i in range(len(images)):
            image = images[i]
            img = image.reshape((1, image.shape[0], image.shape[1], image.shape[2])) 

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 4).output)
            Flatten = keras_function([img])

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 3).output)
            FC7 = keras_function([img])

            keras_function = K.function([model.get_layer(index=0).input], model.get_layer(index=l - 2).output)
            FC6 = keras_function([img])

            
            Flatten_list.append(Flatten[0])
            FC7_list.append(FC7[0])
            FC6_list.append(FC6[0])
    pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=len(images)))])
    flatten_new=pcaFuse(Flatten_list,pipeline)
    fc7_new=pcaFuse(FC7_list,pipeline)
    fc6_new=pcaFuse(FC7_list,pipeline)
    t=octave.fuse(flatten_new,fc7_new,fc6_new,missing_labels_en)

    return t

In [12]:

def pcaFuse(layer,pipeline):
    layer=pipeline.fit_transform(layer)
    return layer

In [13]:
img = os.path.join(dir,'content','media','images','suraj.jpg')
faces = detect_face(img)
face = faces[0]
aligned_img =  fa.align(cv2.imread(img), face['keypoints']['left_eye'], face['keypoints']['right_eye'])

In [ ]:
encode = models.Sequential()
encode.add(model)
encode.add(layers.Flatten())
encode.add(layers.Dense(512, activation='relu'))
encode.add(layers.Dropout(0.5))
encode.add(layers.Dense(len(set(missing_labels))-1, activation='softmax'))

In [ ]:
img = cv2.equalizeHist(cv2.cvtColor(aligned_img, cv2.COLOR_BGR2GRAY))
            
img= cv2.resize(img,(224,224))
img2 = cv2.merge((img,img,img))
img = img_to_array(img2)
i=np.array([img])
predict_img=encode.predict(i)

In [ ]:
images = preprocess(missing_imgs,aligned_img)


In [ ]:
encoded_dataset = feature_extraction(images)

In [189]:
min=0
individual_rank={}
i=0
for c in encoded_dataset:
    dist = np.linalg.norm(c-predict_img)
    if missing_labels[i] not in individual_rank:
        individual_rank[missing_labels[i]]=dist
    elif missing_labels[i] in individual_rank:
        individual_rank[missing_labels[i]]+=dist
    #print('Answer',missing_labels[i], dist)     
    i+=1
print(individual_rank)  
for k,v in individual_rank.items():
    if v>min :
        min=v
        result=k
print(result,min)

{'SanjanaTrivari_2007-03-04': 3.7673311978013517, 'AndyLilith_2002-09-09': 4.147830884606251, 'AshinaRai_2013-09-04': 4.414126666682539, 'ManishPaul_2019-09-22': 4.132553413721718, 'AdamSingh_2016-02-28': 4.211225650429949, 'SushaNair_2022-02-03': 3.5266557111122627, 'SurajSinha_2022-12-05': 5.028931715920562, 'KeerthanaK_2022-05-22': 3.0430214524079258, 'AnsuiyaSharma_2021-05-10': 3.673277103799743, 'KusumRamachandra_2013-03-04': 3.7321973126991486}
SurajSinha_2022-12-05 5.028931715920562


In [ ]:
suraj
Answer SushaNair_2022-02-03 1.1133775801637613
Answer SushaNair_2022-02-03 1.0678103711951261
Answer SushaNair_2022-02-03 1.0139145482510632

Answer SurajSinha_2022-12-05 1.2290534424790167
Answer SurajSinha_2022-12-05 1.1925012967695319
Answer SurajSinha_2022-12-05 1.1557163813794402

In [ ]:
susha
Answer SushaNair_2022-02-03 1.4152642441897598
Answer SushaNair_2022-02-03 1.3460243265930465
Answer SushaNair_2022-02-03 1.3142974229930382

Answer SurajSinha_2022-12-05 0.9846364009483524
Answer SurajSinha_2022-12-05 0.9431183644524782
Answer SurajSinha_2022-12-05 0.9122814727067913

In [ ]:
Answer ManishPaul_2019-09-22 1.195047598669227
Answer ManishPaul_2019-09-22 1.1235340147159012
Answer ManishPaul_2019-09-22 1.0894295683844024
Answer ManishPaul_2019-09-22 1.015105884942095

Answer SushaNair_2022-02-03 1.2939771655940806
Answer SushaNair_2022-02-03 1.2357712414039108
Answer SushaNair_2022-02-03 1.1970747858401418

Answer SurajSinha_2022-12-05 0.9229741650187925
Answer SurajSinha_2022-12-05 0.9010818062271367
Answer SurajSinha_2022-12-05 0.8459668290123301